<img src="https://www.10botics.com/images/logo_website_700x200-300x86.png" width="300"/>

# 🤖 第二步：訓練你的 AI 裁判 (Teachable Machine)

在這個步驟中，我們要教電腦看懂什麼是「剪刀、石頭、布」。我們不需要寫複雜的程式，只需要使用 Google 的工具就能完成！

---
## 📸 步驟 1: 建立資料集資料夾

我們需要建立 4 個房間（資料夾）來存放照片：
* `dataset/Rock`
* `dataset/Paper`
* `dataset/Scissors`
* `dataset/None`

還記得在 `4.3` 課程中，我們學過如何用指令建立資料夾嗎？
請使用 `!mkdir -p` 指令，一次把這四個資料夾建好！

**提示：** 你可以在一行指令後面接多個路徑，用空白隔開。

In [ ]:
# --- 練習：使用指令建立資料夾 ---
# 請在下方填寫指令，建立上述四個資料夾

# !mkdir -p dataset/Rock __________________________________________

### 💡 解答 (點擊展開)

In [ ]:
# 解答：一次建立所有資料夾
!mkdir -p dataset/Rock dataset/Paper dataset/Scissors dataset/None

# 檢查一下是否建立成功
!ls -R dataset

### 定義拍照函式 (Define Function)

還記得第 4.3 章嗎？我們定義過一個函式來幫忙拍照。
這裡我們再寫一次，並加入 **「指定儲存資料夾」** 的功能。

In [ ]:
import cv2
import time
from picamera2 import Picamera2

# 定義函式：傳入相機物件、資料夾名稱、檔案名稱
def capture_and_save(camera, folder, filename):
    
    # 1. 擷取
    frame = camera.capture_array()
    
    # 2. 轉檔 (RGB -> BGR)
    # Picamera2 預設是 RGB，存檔需要轉回 BGR
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGBA2BGR)
    
    # 3. 翻轉 (根據你的鏡頭方向修改，這裡是左右鏡像)
    frame_flipped = cv2.flip(frame_bgr, 1)
    
    # 4. 組合路徑並存檔
    save_path = f"{folder}/{filename}"
    cv2.imwrite(save_path, frame_flipped)
    
    print(f"已儲存: {save_path}")

print("✅ 拍照小幫手準備好了！")

## 📸 步驟 2: 開始收集

現在程式碼變得很簡單了！我們使用 `for` 迴圈搭配剛剛寫好的小幫手。

**任務：收集 50 張「石頭」的照片**
1. 執行程式。
2. 看到倒數提示時，擺出石頭手勢。
3. 程式會自動每隔 0.5 秒拍一張。

In [ ]:
import cv2
import time
import ipywidgets as widgets
from IPython.display import display
from picamera2 import Picamera2

# --- 設定參數 ---
CATEGORY = "Rock"       # 類別名稱 (記得改成 Paper, Scissors, None)
FOLDER = "dataset/Rock" # 存到哪裡 (記得對應修改)
COUNT = 50              # 拍幾張
DELAY = 0.5             # 間隔幾秒

# --- 建立顯示介面 ---
# 1. 影像顯示區
image_widget = widgets.Image(format='jpeg', width=640, height=480)
# 2. 進度條 (0 ~ COUNT)
progress_bar = widgets.IntProgress(value=0, min=0, max=COUNT, description='進度:')
# 3. 狀態標籤
status_label = widgets.Label(value=f"準備收集 {CATEGORY}... 3秒後開始！")

# 顯示所有元件
display(widgets.VBox([image_widget, progress_bar, status_label]))

# 倒數 3 秒
time.sleep(3)
status_label.value = "開始拍照！"

try:
    with Picamera2() as picam2:
        # 設定相機
        config = picam2.create_preview_configuration(main={"size": (640, 480)})
        picam2.configure(config)
        picam2.start()
        
        for i in range(COUNT):
            # 1. 抓取影像
            frame = picam2.capture_array()
            
            # 2. 轉檔 (RGB -> BGR) 
            # 修正：記得處理顏色和翻轉
            frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGBA2BGR)
            frame_flipped = cv2.flip(frame_bgr, 1) # 1=左右鏡像
            
            # 3. 存檔 (存下高畫質原圖)
            filename = f"{CATEGORY}_{i}.jpg"
            save_path = f"{FOLDER}/{filename}"
            cv2.imwrite(save_path, frame_flipped)
            
            # 4. 更新畫面顯示 (這就是你要的功能！)
            # 在畫面上加字，讓你知道現在拍的是這張
            cv2.putText(frame_flipped, f"{i+1}/{COUNT}", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            _, buffer = cv2.imencode('.jpeg', frame_flipped, [int(cv2.IMWRITE_JPEG_QUALITY), 50])
            image_widget.value = buffer.tobytes()
            
            # 5. 更新進度條與文字
            progress_bar.value = i + 1
            status_label.value = f"已儲存: {filename}"
            
            # 休息一下
            time.sleep(DELAY)
            
    status_label.value = f"🎉 {CATEGORY} 收集完成！共 {COUNT} 張。"
    print("完成！")

except Exception as e:
    status_label.value = f"❌ 錯誤: {e}"
    print(e)

### 收集其他手勢

修改上面的程式碼，修改 `CATEGORY` 和 `FOLDER`，分別收集：
* **Paper** (布)
* **Scissors** (剪刀)
* **None** (背景/沒人)

**提示：** 收集 `None` (背景) 時，人要離開鏡頭喔！

## 📦 步驟 3: 下載 (Package & Download)

照片收集完成了！現在我們要把它們下載到電腦，再到teachable machine網站進行training。

**操作步驟：**
1.  執行下方的指令，將 `dataset` 資料夾壓縮成 `my_dataset.zip`。
2.  看到左側檔案列表出現 `my_dataset.zip`。
3.  對著它按 **右鍵** -> **Download (下載)**。
4.  接著你就可以去 Teachable Machine 網站上傳這個壓縮檔了！

In [ ]:
# -r 代表遞迴 (包含資料夾裡面的所有東西)
# 指令格式: !zip -r [產生的檔名] [要壓縮的資料夾]

!zip -r my_dataset.zip dataset

print("✅ 打包完成！請在左側尋找 'my_dataset.zip' 並下載。")

## 🤖 步驟 4：訓練你的 AI 裁判 (Teachable Machine)

在這個步驟中，我們要教電腦看懂什麼是「剪刀、石頭、布」。我們不需要寫複雜的程式，只需要使用 Google 的工具就能完成！

---

## 🟢 1. 建立專案
1.  開啟瀏覽器，前往 [Teachable Machine 網站](https://teachablemachine.withgoogle.com/train)。
2.  點選 **"Image Project" (影像專案)**。
3.  選擇 **"Standard Image Model" (標準影像模型)**。
<br>
    <img src="./images/create_new_project.png" width="700" />
---

## 📷 2. 上傳數據 (Upload Data)

我們剛剛已經把照片打包成 `my_dataset.zip` 並下載到電腦了。現在要把這些照片上傳到 Teachable Machine。

### 2.1：解壓縮
首先，請在你電腦上找到 `my_dataset.zip`，按右鍵選擇 **解壓縮 (Unzip/Extract)**。你會看到一個包含 `Rock`, `Paper`, `Scissors`, `None` 四個資料夾的目錄。

### 2.2：設定類別與上傳
請依照以下順序設定四個類別：

1.  **修改類別名稱**：將 "Class 1" 改名為 **"Rock"**。
2.  **上傳照片**：
    * 點擊 **"Upload" (上傳)**。
    <br>
    <img src="./images/upload_image.png" width="700" />
    * 選擇 **"Choose images from your files"**。
    <br>
    <img src="./images/choose_images_from_your_files.png" width="700" />
    * 進入你電腦剛解壓縮的 `Rock` 資料夾，**全選** 裡面的照片並開啟。
    <br>
    <img src="./images/select_rock_image.png" width="700" />
    <br>
    <img src="./images/upload_rock__image.png" width="700" />

3.  **重複動作**：
    * 將 "Class 2" 改名為 **"Paper"**，並上傳 `Paper` 資料夾的照片。
    * 點擊下方的 **"Add a class" (新增類別)**，建立 **"Scissors"** 並上傳照片。

### 2.3：加入 "None" (背景)
這一步最重要！我們要告訴 AI 「什麼都不做」長什麼樣子。

1.  再新增一個類別，命名為 **"None"** (或 Background)。
2.  上傳 `None` 資料夾裡面的背景照片。

**完成後，你的畫面應該要有這 4 個類別，並且都有照片：**

---

## 🧠 3. 訓練模型 (Training)
1.  按下中間的 **"Train Model" (訓練模型)** 按鈕。
2.  **請勿關閉或切換分頁**，這需要一點時間（約 1~2 分鐘）。
<br>
    <img src="./images/train_model.png" width="700" />
3.  訓練完成後，在右側的 "Preview" (預覽) 視窗試玩看看，確認 AI 能準確辨認你的手勢。
<br>
    <img src="./images/test_model.png" width="700" />

---

## 📥 4. 匯出模型 (Export Model)
我們要把訓練好的「大腦」下載下來，準備放進 Raspberry Pi。

1.  點擊右側的 **"Export Model"** 按鈕。
2.  上方分頁選擇 **"TensorFlow Lite"**。
3.  下方格式選擇 **"Quantized" (量化)**。
    * *為什麼？* 量化模型體積小、速度快，最適合 Raspberry Pi。
4.  點擊 **"Download my model"**。
<br>
    <img src="./images/export_model.png" width="700" />

---

## ✅ 5. 檢查檔案
下載並解壓縮後，請確認你有以下兩個檔案：
1.  `model.tflite` (模型本體)
2.  `labels.txt` (標籤檔，告訴程式 0 1 2 3 是甚麼)

<hr/>

## Congratulation! You have finished this chapter.

This jupyter notebook is created by 10Botics. <br>
For permission to use in school, please contact info@10botics.com <br>
All rights reserved. 2024.